# TensorFlow Mechanics 101

- This tutorial is meant as a companion to the code [here](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist/)
- The goal of this tutorial is to show how to use TensorFlow to train and evaluate a simple feed-forward neural network for handwritten digit classification using the (classic) MNIST data set. 

- [`mnist.py`](https://www.tensorflow.org/code/tensorflow/examples/tutorials/mnist/mnist.py), the code for making a fully-connected MNIST model
- [`fully_connected_feed.py`](https://www.tensorflow.org/code/tensorflow/examples/tutorials/mnist/fully_connected_feed.py), the main code to train the built MNIST model against the downloaded dataset using a feed dictionary.

In [14]:
import math
import time
import random
from os import getcwd
from os.path import join

import pandas as pd
import tensorflow as tf

## Using a Custom e-rater Data-set

In [54]:
# Download test_data_revised.zip (in email since it can't be shared) and
# save it somewhere, preferably in the directory in which this notebook is
# stored. If it is somewhere else, just make sure to pass in the path when
# this function is used.

def read_praxis_data(macro_or_micro="macro",
                     validation_set=True,
                     data_path=join(getcwd(), "test_data_revised")):
    """
    Read in data from a directory with the following files:
    testing_macro.csv and training_macro.csv.

    Either read in the "macro" files or the "micro" files.

    Returns 1) training IDs, features, and labels,
            2) test IDs, features, and labels, and
            3) validation IDs, features, and labels (if `validation_set` is
               False, the test test set will contain all data originally in
               the "testing" file).
    """
    
    train_data_path = join(data_path,
                           "training_{}.csv"
                           .format(macro_or_micro if macro_or_micro == "macro"
                                   else "micro_revised"))
    train_data = pd.read_csv(train_data_path, dtype={'appointment_id': str})
    train_labels = train_data['H1'].apply(lambda x: x - 1)
    train_features = train_data[[a for a in train_data.columns
                                 if a not in ['appointment_id', 'H1']]]
    train_ids = train_data['appointment_id']

    test_data_path = join(data_path,
                          "testing_{}.csv"
                          .format(macro_or_micro if macro_or_micro == "macro"
                                  else "micro_revised"))
    test_data = pd.read_csv(test_data_path, dtype={'appointment_id': str})
    if len(test_data) % 100:
        test_data = test_data.head(len(test_data) - len(test_data) % 100)
        
    test_labels = test_data['H1'].apply(lambda x: x - 1)
    test_features = test_data[[a for a in test_data.columns
                               if a not in ['appointment_id', 'H1']]]
    test_ids = test_data['appointment_id']

    val_features = None
    val_labels = None
    val_ids = None
    if validation_set:
        test_amount = len(test_features) - 1000
        val_features = test_features.head(1000)
        test_features = test_features.tail(test_amount)
        val_labels = test_labels.head(1000)
        test_labels = test_labels.tail(test_amount)
        val_ids = test_ids[:1000]
        test_ids = test_ids[1000:]

    return (train_ids,
            train_features,
            train_labels,
            test_ids,
            test_features,
            test_labels,
            val_ids,
            val_features,
            val_labels)

In [49]:
test_data_path = join(getcwd(), "test_data_revised",
                          "testing_{}.csv"
                          .format("micro_revised"))

In [50]:
test_data = pd.read_csv(test_data_path, dtype={'appointment_id': str})

In [51]:
len(test_data) % 100

50

In [53]:
len(test_data.head(len(test_data) - len(test_data) % 100))

2700

In [55]:
# Choose "micro" or "macro". This will change the types of features we're
# using. There are 220 "micro" features in total while thre are less than
# 10 macro features.
dataset_type = "micro"

In [56]:
# Read in data
(train_ids, train_features, train_labels,
 test_ids, test_features, test_labels,
 validation_ids, validation_features, validation_labels) = \
    read_praxis_data(macro_or_micro=dataset_type, validation_set=True)

### Data and Data Shape

In [57]:
print("Shape of data:\n\tTraining: {}\n\tValidation: {}\n\tTest: {}"
      .format(train_features.shape,
              None if validation_features is None else validation_features.shape,
              test_features.shape))
print("Shape of labels data:\n\tTraining: {}\n\tValidation: {}\n\tTest: {}"
      .format(train_labels.shape,
              None if validation_labels is None else validation_labels.shape,
              test_labels.shape))

Shape of data:
	Training: (4000, 220)
	Validation: (1000, 220)
	Test: (1750, 220)
Shape of labels data:
	Training: (4000,)
	Validation: (1000,)
	Test: (1750,)


In [19]:
# What the features look like
train_features.head()

,ABSTRACT_NOUNS_LESS127PLUS4_TYPE,ACA_DOWN_CNT_NORM,ADV_CAUSATIVE,ADV_COND,ADV_FOCUS,ADV_INSTANTIATION,ADV_NEG,ADV_PLACE_NORM,ADV_PLACEPRO,ADV_TEMP,...,WH,WH_ADVERB,WH_DETERMINER,WH_POSSESSIVE,WH_PRONOUN,WORD_CNT,WORDLEN,WORDS,WORDS2,WRDS_ENCL_QUOT
0,0.931822,0.000000,0.000000,0.0,1.018449,0.0,0.460416,0.823499,0.931822,0.931822,...,1.018449,0.678845,1.018449,0.0,0.823499,530,4,526,276676,0
1,1.125961,0.405789,0.405789,0.0,0.751031,0.0,0.405789,0.000000,0.611850,0.751031,...,1.255310,0.856268,0.751031,0.0,1.072587,647,4,633,400689,0
2,1.239548,0.483586,0.000000,0.0,0.706716,0.0,0.706716,0.000000,0.483586,0.000000,...,1.239548,0.706716,0.853392,0.0,1.050184,489,4,481,231361,0
3,0.779966,0.000000,0.000000,0.0,1.043372,0.0,0.000000,1.043372,1.043372,0.779966,...,1.324293,1.043372,0.000000,0.0,1.043372,199,4,196,38416,0
4,1.485697,0.000000,0.493342,0.0,0.975738,0.0,0.865844,1.063364,0.718363,0.493342,...,0.975738,0.493342,0.493342,0.0,0.718363,473,4,471,221841,0


In [20]:
# Labels are on a 0 to 5 scale (scores 1 to 6)
train_labels[:10]

0    3.0
1    4.0
2    3.0
3    2.0
4    4.0
5    3.0
6    2.0
7    1.0
8    3.0
9    3.0
Name: H1, dtype: float64

In [21]:
train_labels.value_counts()

3.0    2086
2.0    1279
4.0     405
1.0     178
5.0      41
0.0      11
Name: H1, dtype: int64

In [27]:
test_labels.value_counts()

3.0    889
2.0    576
4.0    196
1.0     70
5.0     15
0.0      4
Name: H1, dtype: int64

In [28]:
validation_labels.value_counts()

3.0    506
2.0    337
4.0    107
1.0     42
5.0      6
0.0      2
Name: H1, dtype: int64

In [22]:
# Define some parameters
log_dir_path = join(getcwd(), "logs")
learning_rate = 0.01
max_steps = 2000
if dataset_type == "macro":
    hidden1 = 64
    hidden2 = 32
    hidden3 = 16
    NUM_FEATURES = 9
else:
    hidden1 = 512
    hidden2 = 256
    hidden3 = 128
    NUM_FEATURES = 220
batch_size = 100
NUM_CLASSES = 6

## Functions Based on `mnist.py`

In [31]:
def inference(inputs, num_features, num_classes, hidden1_units,
              hidden2_units, hidden3_units):
    """
    Build a model on the inputs up to where it may be used for
    inference.

    Args:
        inputs: Placeholder for input data samples.
        num_features: Number of features in input data.
        num_classes: Number of classes/score labels.
        hidden1_units: Size of the first hidden layer.
        hidden2_units: Size of the second hidden layer.
        hidden3_units: Size of the third hidden layer.

    Returns:
        softmax_linear: Output tensor with the computed logits.
    """

    # Hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
              tf.truncated_normal([num_features, hidden1_units],
                                  stddev=1.0 / math.sqrt(float(num_features))),
              name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(inputs, weights) + biases)

    # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

    # Hidden 3
    with tf.name_scope('hidden3'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, hidden3_units],
                                stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden3_units]),
                             name='biases')
        hidden3 = tf.nn.relu(tf.matmul(hidden2, weights) + biases)

    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden3_units, num_classes],
                                stddev=1.0 / math.sqrt(float(hidden3_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([num_classes]),
                             name='biases')
        logits = tf.matmul(hidden3, weights) + biases

    return logits


def loss(logits, labels):
    """
    Calculates the loss from the logits and the labels.

    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size].

    Returns:
        loss: Loss tensor of type float.
    """

    labels = tf.to_int64(labels)
    cross_entropy = \
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                       logits=logits,
                                                       name='xentropy')

    return tf.reduce_mean(cross_entropy, name='xentropy_mean')


def training(loss, learning_rate):
    """
    Sets up the training Ops.
    
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable
    variables.
    
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.

    Args:
        loss: Loss tensor, from loss().
        learning_rate: The learning rate to use for gradient descent.

    Returns:
        train_op: The Op for training.
    """

    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', loss)

    # Create the gradient descent optimizer with the given learning
    # rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single
    # training step.
    train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """
    Evaluate the quality of the logits at predicting the label.

    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size], with values in the
                range [0, NUM_CLASSES).

    Returns:
        A scalar int32 tensor with the number of examples (out of
        batch_size) that were predicted correctly.
    """

    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)

    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [32]:
# From fully_connected_feed.py
def fill_feed_dict(features, labels, inputs_pl, labels_pl, batch_size):
    """
    Fills the feed_dict for training the given step.

    A feed_dict takes the form of:
    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }

    Args:
        data_set: The set of features and labels.
        inputs_pl: The input data placeholder.
        labels_pl: The input labels placeholder.
        batch_size: Size of each randomly-selected batch.

    Returns:
        feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch_size` samples.
    sample = random.sample(range(len(features)), k=batch_size)
    inputs_feed = features.ix[sample]
    labels_feed = labels.ix[sample]
    
    
    feed_dict = {
        inputs_pl: inputs_feed,
        labels_pl: labels_feed,
    }

    return feed_dict


def do_eval(sess, eval_correct, inputs_placeholder, labels_placeholder,
            features, labels):
    """
    Runs one evaluation against the full epoch of data.

    Args:
        sess: The session in which the model has been trained.
        eval_correct: The Tensor that returns the number of correct
                      predictions.
        inputs_placeholder: The input data placeholder.
        labels_placeholder: The labels placeholder.
        data_set: The set of images and labels to evaluate, from
                  input_data.read_data_sets().
    """

    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = features.shape[0] // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in range(steps_per_epoch):
        feed_dict = fill_feed_dict(features, labels,
                                   inputs_placeholder,
                                   labels_placeholder,
                                   batch_size)
        logit_output, true_cnt = sess.run([logits, eval_correct],
                                          feed_dict=feed_dict)
        true_count += true_cnt

    precision = float(true_count) / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
          (num_examples, true_count, precision))

In [34]:
# Tell TensorFlow that the model will be built into the default Graph.
with tf.Graph().as_default():

    # Generate placeholders for the input feature data and labels.
    inputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                           NUM_FEATURES))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

    # Build a Graph that computes predictions from the inference model.
    logits = inference(inputs_placeholder,
                       NUM_FEATURES,
                       NUM_CLASSES,
                       hidden1,
                       hidden2,
                       hidden3)

    # Add to the Graph the Ops for loss calculation.
    loss_ = loss(logits, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
    train_op = training(loss_, learning_rate)

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct = evaluation(logits, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()

    # Add the variable initializer Op.
    init = tf.global_variables_initializer()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(log_dir_path, sess.graph)

    # And then after everything is built:

    # Run the Op to initialize the variables.
    sess.run(init)

    # Start the training loop.
    for step in range(max_steps):
        start_time = time.time()

        # Fill a feed dictionary with the actual set of images and labels
        # for this particular training step.
        feed_dict = fill_feed_dict(train_features, train_labels,
                                   inputs_placeholder,
                                   labels_placeholder,
                                   batch_size)

        # Run one step of the model.  The return values are the activations
        # from the `train_op` (which is discarded) and the `loss` Op.  To
        # inspect the values of your Ops or variables, you may include them
        # in the list passed to sess.run() and the value tensors will be
        # returned in the tuple from the call.
        _, loss_value = sess.run([train_op, loss_],
                                 feed_dict=feed_dict)

        duration = time.time() - start_time

        # Write the summaries and print an overview fairly often.
        if step % 100 == 0:

            # Print status to stdout.
            print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
            # Update the events file.
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
        
        # Save a checkpoint and evaluate the model periodically.
        if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
            checkpoint_file = join(log_dir_path, 'model.ckpt')
            saver.save(sess, checkpoint_file, global_step=step)

            # Evaluate against the training set.
            print('Test Data Eval:')
            do_eval(sess,
                    eval_correct,
                    inputs_placeholder,
                    labels_placeholder,
                    train_features, train_labels)

            # Evaluate against the validation set.
            print('Validation Data Eval:')
            do_eval(sess,
                    eval_correct,
                    inputs_placeholder,
                    labels_placeholder,
                    validation_features, validation_labels)

            # Evaluate against the test set.
            print('Test Data Eval:')
            do_eval(sess,
                    eval_correct,
                    inputs_placeholder,
                    labels_placeholder,
                    test_features, test_labels)

Step 0: loss = 1.79 (0.013 sec)
Step 100: loss = 1.59 (0.009 sec)
Step 200: loss = 1.44 (0.011 sec)
Step 300: loss = 1.45 (0.009 sec)
Step 400: loss = 1.38 (0.009 sec)
Step 500: loss = 1.28 (0.010 sec)
Step 600: loss = 1.34 (0.009 sec)
Step 700: loss = 1.31 (0.011 sec)
Step 800: loss = 1.14 (0.010 sec)
Step 900: loss = 1.26 (0.010 sec)
Test Data Eval:
  Num examples: 4000  Num correct: 2054  Precision @ 1: 0.5135
Validation Data Eval:
  Num examples: 1000  Num correct: 506  Precision @ 1: 0.5060
Test Data Eval:


InvalidArgumentError: targets[1] is out of range
	 [[Node: InTopK = InTopK[T=DT_INT32, k=1, _device="/job:localhost/replica:0/task:0/cpu:0"](softmax_linear/add, _recv_Placeholder_1_0)]]

Caused by op 'InTopK', defined at:
  File "/home/mmulholland/Documents/learning/tensorflow/tf_env/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mmulholland/Documents/learning/tensorflow/tf_env/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mmulholland/Documents/learning/tensorflow/tf_env/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/mmulholland/Documents/learning/tensorflow/tf_env/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/mmulholland/Documents/learning/tensorflow/tf_env/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.insta